In [ ]:
import pandas as pd

### Preparation of Dataframes

In [ ]:
df1 = pd.read_csv("organizations_processed_1.csv")
df2 = pd.read_csv("organizations_processed_2.csv")
df3 = pd.read_csv("organizations_processed_3.csv")
org_df = df1.append(df2)
org_df = org_df.append(df3)
org_df = org_df.reset_index()
org_df = org_df.drop(columns=["index"])

df1 = pd.read_csv("people_processed_1.csv")
df2 = pd.read_csv("people_processed_2.csv")
df3 = pd.read_csv("people_processed_3.csv")
df4 = pd.read_csv("people_processed_4.csv")
df5 = pd.read_csv("people_processed_5.csv")
ppl_df = df1.append(df2)
ppl_df = ppl_df.append(df3)
ppl_df = ppl_df.append(df4)
ppl_df = ppl_df.append(df5)
ppl_df = ppl_df.reset_index()
ppl_df = ppl_df.drop(columns=["index"])

In [ ]:
acq_df = pd.read_csv("acquisitions_processed.csv")
fund_df = pd.read_csv("funds_processed.csv") #investors' investment funds
fund_rd_df = pd.read_csv("funding_rounds_processed.csv") # each funding round in the dataset
ipo_df = pd.read_csv("ipos_processed.csv") 
csum_df = pd.read_csv("checksum_processed.csv") # The sum of data present in each csv file (the number of rows in each dataframe)
deg_df = pd.read_csv("degrees.csv") #people's education background
cate_gp_df = pd.read_csv("category_groups_processed.csv") #Company name with its group

df1 = pd.read_csv("event_appearances_processed_1.csv")
df2 = pd.read_csv("event_appearances_processed_2.csv")
event_app_df = df1.append(df2)
event_app_df= event_app_df.reset_index()
event_app_df= event_app_df.drop(columns=["index"]) # Event participation details

event_df= pd.read_csv("events_processed.csv") #Event details

In [ ]:
df1 = pd.read_csv("jobs_processed_1.csv")
df2 = pd.read_csv("jobs_processed_2.csv")
df3 = pd.read_csv("jobs_processed_3.csv")
df4 = pd.read_csv("jobs_processed_4.csv")
df5 = pd.read_csv("jobs_processed_5.csv")
job_df = df1.append(df2)
job_df= job_df.append(df3)
job_df= job_df.append(df4)
job_df= job_df.append(df5)
job_df= job_df.reset_index()
job_df= job_df.drop(columns=["index"]) #all job and advisory roles

invest_partner_df= pd.read_csv("investment_partners_processed.csv") #Partners who are responsible for their firm's investments
investor_df= pd.read_csv("investors_processed.csv") #Active investors 
org_parent_df= pd.read_csv("org_parents_processed.csv") #Mapping between parent organizations and subsidaries

#All dataframes are now ready, org_df is the master dataframe
org_df

### Handling Naming Issues, and dropping some overlapped columns

In [ ]:
# Some prefixes are needed due to duplicate coulmn names with other csv
# Some repeated columns existing info in other dataframes can be deleted

acq_df = acq_df.drop(columns=['acquiree_name', 'acquiree_country_code', 'acquiree_region', 'acquiree_city', 'rank'])
acq_df = acq_df.add_prefix('acquisitions_')
fund_df = fund_df.add_prefix('funds_')
ipo_df = ipo_df.add_prefix('ipo_')
fund_rd_df = fund_rd_df.drop(columns=['rank'])
fund_rd_df = fund_rd_df.add_prefix('fund_rd_')
cate_gp_df = cate_gp_df.drop(columns=['name'])
cate_gp_df = cate_gp_df.add_prefix('cat_')
job_df = job_df.add_prefix('job_')
org_parent_df = org_parent_df.add_prefix('parent_org_')

laed_investor_df = investor_df.add_prefix('lead_investor_')
partner_df = investor_df.add_prefix('partner_')
investor_df = investor_df.add_prefix('investor_')
event_df = event_df.add_prefix('event_')
event_df = event_df.rename(columns={'event_name': 'event_names'})
event_app_df = event_app_df.drop(columns=['uuid'])

deg_df = deg_df.drop(columns=['created_at','updated_at', 'type'])
deg_df = deg_df.add_prefix('degree_')
ppl_df = ppl_df.drop(columns=['created_at','updated_at'])
ppl_df = ppl_df.add_prefix('personal_')

### Firstly, handle investor, partners and fundings. As a partner is also an investor himself, two joinings are needed

In [ ]:
# Joining Investor and Partners

#investor_df = investor_df.drop(columns=['investor_name', 'investor_country_code', 'investor_region', 'investor_city'])

invest_join_df = invest_partner_df.set_index('partner_uuid').join(partner_df.set_index('partner_uuid'))
invest_join_df = invest_join_df.set_index('investor_uuid').join(investor_df.set_index('investor_uuid'))
invest_join_df.drop(columns=['uuid'], inplace = True)

In [ ]:
# Joining investor and funding rounds
fund_rd_df = fund_rd_df.set_index('fund_rd_uuid').join(invest_join_df.set_index('funding_round_uuid'))

# Joining lead investor and funding rounds
fund_rd_df = laed_investor_df.set_index('lead_investor_uuid').join(fund_rd_df.set_index('fund_rd_lead_investor_uuids'))

### Next, we handle the event. Link the event participants to the events

In [ ]:
# Joining event and event_app
event_join_df = event_df.set_index('event_uuid').join(event_app_df.set_index('event_uuid'))

event_join_df.drop(columns =['permalink', 'created_at', 'updated_at','event_name'], inplace = True)

people_event_df = event_join_df.loc[event_join_df['participant_type'] == 'person']
org_event_df = event_join_df.loc[event_join_df['participant_type'] == 'organization']

### After sorting out all dataframes that are replated to "people entity", we are ready to build up a large dataframe that consists of people_uuid as index

In [ ]:
#Joining People and Degrees
ppl_join = ppl_df.set_index('personal_uuid').join(deg_df.set_index('degree_person_uuid'))

In [ ]:
#Joining People and People Event Participant 
ppl_join = ppl_join.join(people_event_df.set_index('participant_uuid'))

In [ ]:
#Add prefix to avoid overlap of column names
ppl_join = ppl_join.add_prefix('person_')

### Now, it is time to handle organizations, put org_uuid as index

In [ ]:
#Joining Organizations and IPO
org_join = org_df.set_index('uuid').join(ipo_df.set_index('ipo_org_uuid'))

In [ ]:
#Joining Organizations and funds
org_join = org_join.join(fund_df.set_index('funds_entity_uuid'))

In [ ]:
#Joining Organizations and fund_rounds
org_join = org_join.join(fund_rd_df.set_index('fund_rd_org_uuid'))

In [ ]:
#Joining Organizations and acquisitions
org_join = org_join.join(acq_df.set_index('acquisitions_acquiree_uuid'))

In [ ]:
#Joining Organizations and categories
org_join = org_join.join(cate_gp_df.set_index('cat_uuid'))

In [ ]:
#Joining Organizations and its parents, if any
org_join = org_join.join(org_parent_df.set_index('parent_org_uuid'))

In [ ]:
#Joining Organizations and Organization Event Participant
org_join = org_join.join(org_event_df.set_index('participant_uuid'))

### Have a look at the two large dataframes (people and organization)

In [ ]:
#Drop away emtpy columns to improve efficiency
ppl_join.dropna(how = 'all', axis = 1, inplace = True)
#Drop away emtpy columns to improve efficiency
org_join.dropna(how = 'all', axis = 1, inplace = True)

In [ ]:
ppl_join.info()

In [ ]:
org_join.info()

In [ ]:
#Drop away columns that are without any use to improve efficiency
org_join = org_join.drop(columns = ['permalink', 'created_at', 'updated_at', 'ipo_uuid', 'ipo_org_name', 'funds_uuid', \
                'acquisitions_uuid', 'parent_org_parent_uuid', 'ipo_stock_symbol', 'participant_name','event_venue_name', \
                'event_short_description', 'event_description'])

### Finally, it is about the linkage between people and organization, through "job"

In [ ]:
#Join Job with People
job_join = job_df.set_index('job_person_uuid').join(ppl_join)

In [ ]:
#Drop away columns that are without any use to improve efficiency
job_join = job_join.drop(columns=['job_uuid', 'job_permalink', 'job_created_at', 'job_updated_at', 'person_participant_name',\
                                 'person_degree_uuid', 'person_degree_institution_uuid', 'person_degree_person_name','person_event_venue_name',\
                                 'person_event_short_description', 'person_event_description'])
job_join = job_join.rename(columns={'job_person_name': 'person_name','person_event_event_roles':'person_event_roles',\
                                   'event_event_roles':'event_roles', 'event_names':'event_name', 'person_event_names': 'person_event_name'})

In [ ]:
#Make sure uuid is not lost if export to CSV in the next cell
org_join['uuid'] = org_join.index

In [ ]:
"""
#If necessary, export here first to avoid the runninng codes dead due to insufficient memory

job_join.to_csv('job_join.csv')
org_join.to_csv('org_join.csv')  

#Refer to Joining_Dataframes_Cont'd for further running after exporting into csv files
"""

In [ ]:
#Since memory may not be able to handle all at once, split it into multiple dataframes before joining is safer
job_join_arr = []
interval = int(len(job_join) / 10)
for i in range(10):
    job_join_part = job_join.iloc[interval*i : interval*(i+1)]
    job_join_arr.append(job_join_part)

In [ ]:
#Take around half hour to run this cell

#Join Organization with Job
for i in range(10):
    org_join = org_join.set_index('uuid').join(job_join_arr[i].set_index('job_org_uuid'))

In [ ]:
#Drop away emtpy columns to simplify the final dataframe
org_join.dropna(how = 'all', axis = 1, inplace = True)

### Have a look at the joint dataframe

In [ ]:
#Take around 15 minutes to run this cell

#Let's see the info of the final DataFrame
org_join.info()

In [ ]:
#The final DataFrame
org_join

In [ ]:
"""
#Export out as CSV, the file is around 60GB
#org_join.to_csv('org_final_joined.csv')

#Use the following code to zip before exporting
compression_opts = dict(method = 'zip', archive_name = 'org_final_joined.csv')  
org_join.to_csv('org_final_joined.zip', compression=compression_opts) 
"""

In [ ]:
"""
# Problem 1: Is the following items the same as organization itself? Can we come up a way to check? If yes, we can drop all duplications to simplify our df

# e.g.
#'ipo_country',vs 'fund_rd_country', vs 'country_code'
#'ipo_region', vs 'fund_rd_region', vs 'state_code'
#'ipo_city', vs 'fund_rd_city', vs 'city'
#'person_personal_rank', vs 'personal_event_rank'
#'total_funding_usd', vs 'total_funding', vs 'funds_raised_amount_usd' 
#'category_list', vs 'category_groups_list' (what should we use eaxctly......)
"""
"""
# Problem 2: There are multiple rows for some organization eitites, due to their multiple funding rounds, or multiple people, etc.
#            I plan to keep here first, but we may have to come up with a way to handle this situation
"""
"""
# Problem 3: The dataframe consists of a column 'Acquirer'. I chose to join using acquiriee id instead of acquirier id,
#            due to the intuition that venture companies are more likely to be acquired than to acquire others;
#            However, may need to find a way to handle the situation if there is really a startup acquiring others
"""
"""
# Problem 4: If a startup company has a parent company, do we also need the information of its parent company in
#            our dataframes. My answer is yes and know, coz not sure how much could its parent company affect it;
#            however, after all we value a startup company all by itself. If we have to incorporate its parents' data,
#            how do we valuate its parents? We may have to build up another dataframe, or else it would be too large. 
#            Also, if its parent is an acquirer, the situation will be much more complex.
"""
"""
# Problem 5: Now, all data are raw, many NaN data and are unprocessed, we have to preprocess all the data here at once
#            Also, we have to extract those that are within 5 years and fintech field to be our target data.
"""